In [1]:
import tensorflow as tf
import numpy
import data
import model

learning_rate = .001
grad_clip = 5.
batch_size = 64

Reading files: 08527   
Reading successful!


In [2]:
tf.reset_default_graph()
c = model.classifier(rnn_sizes=[128, 128], fc_sizes=[64, 32], batch_size=batch_size)
feeder = data.batch_pool(batch_size)
batch = next(feeder)

# Loss and opt

In [3]:
labels = tf.placeholder(tf.int32, [c.batch_size])
loss = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=labels, logits=c.logits)
loss = tf.reduce_mean(loss)

optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
gvs = optimizer.compute_gradients(loss)
capped_gvs = [(tf.clip_by_value(grad, -grad_clip, grad_clip), var) for grad, var in gvs]
opt = optimizer.apply_gradients(capped_gvs)

# Train test

In [ ]:
with tf.Session() as sess:
    tf.global_variables_initializer().run()
    for i, batch in enumerate(feeder):
        init_state = sess.run(c.zero_state)
        feed_dict = {
            c.x : batch[0],
            labels : batch[1],
            c.seq_len : batch[2],
            c.init_state:init_state
        }
        
        l, _ = sess.run([loss, opt], feed_dict)
        print(i, l)
        
        if i > 10: break

0 1.3844
1 1.36563
2 1.35482
3 1.30896
4 1.2954
